In [2]:
import pandas as pd

# Check for Missing Values and Duplicates

# Research Categories

In [ ]:
# Path
csv_path2020 = "/Users/mariejatsun/Desktop/MDA Project/cordis-h2020projects-csv/euroSciVoc.csv"
csv_path2024 = "/Users/mariejatsun/Desktop/MDA Project/cordis-HORIZONprojects-csv/euroSciVoc.csv"

# Load CSV's 
df_2020 = pd.read_csv(csv_path2020, delimiter=";", on_bad_lines="skip")
df_2024 = pd.read_csv(csv_path2024, delimiter=";", on_bad_lines="skip")

# Look if column euroSciVocPath exists
if 'euroSciVocPath' not in df_2020.columns or 'euroSciVocPath' not in df_2024.columns:
    raise ValueError("Column 'euroSciVocPath' not found")

# Extract category
df_2020['category2020'] = df_2020['euroSciVocPath'].str.split('/').str[1]
df_2024['category2024'] = df_2024['euroSciVocPath'].str.split('/').str[1]

# Count rows per category
category_counts_2020 = df_2020['category2020'].value_counts().reset_index()
category_counts_2024 = df_2024['category2024'].value_counts().reset_index()

# Choose column names
category_counts_2020.columns = ['category', 'count_2020']
category_counts_2024.columns = ['category', 'count_2024']

# Print 
print("\nNumber of projects per category in 2020:")
print(category_counts_2020.to_string(index=False))

print("\nNumber of projects per category in 2024:")
print(category_counts_2024.to_string(index=False))



Number of projects per category in 2020:
                   category  count_2020
           natural sciences       48421
 engineering and technology       25267
medical and health sciences       17468
            social sciences       16024
                 humanities        4028
      agricultural sciences        2985

Number of projects per category in 2024:
                   category  count_2024
           natural sciences       18075
 engineering and technology        6680
medical and health sciences        5985
            social sciences        5027
                 humanities        1739
      agricultural sciences        1283


# Funding Distribution 

## Comparison Across countries

### In organization.csv: columns ‘ecContribution’, ‘netEcContribution’

In [5]:
# Paths 
csv_path_2020 = "/Users/mariejatsun/Desktop/MDA Project/cordis-h2020projects-csv/organization.csv"
csv_path_2024 = "/Users/mariejatsun/Desktop/MDA Project/cordis-HORIZONprojects-csv/organization.csv"

# Load CSV's
df_2020 = pd.read_csv(csv_path_2020, delimiter=";", on_bad_lines="skip")
df_2024 = pd.read_csv(csv_path_2024, delimiter=";", on_bad_lines="skip")

# Look if required columns exist
required_cols = ['country', 'ecContribution', 'netEcContribution']
for col in required_cols:
    if col not in df_2020.columns or col not in df_2024.columns:
        raise ValueError(f"Column '{col}' not found in one of the datasets")

# Group by country and sum contributions
funding_2020 = df_2020.groupby('country')[['ecContribution', 'netEcContribution']].sum().reset_index()
funding_2024 = df_2024.groupby('country')[['ecContribution', 'netEcContribution']].sum().reset_index()

# Choose column names
funding_2020.columns = ['country', 'ecContribution_2020', 'netEcContribution_2020']
funding_2024.columns = ['country', 'ecContribution_2024', 'netEcContribution_2024']

# Merge datasets on country
merged_funding = pd.merge(funding_2020, funding_2024, on='country', how='outer').fillna(0)

# Print 
print("\nFunding distribution comparison (H2020 vs Horizon Europe):")
print(merged_funding.sort_values(by='ecContribution_2024', ascending=False).to_string(index=False))


/var/folders/xs/sd59f35s4fd7q67ls761c5xm0000gn/T/ipykernel_21906/3966253279.py:6: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2020 = pd.read_csv(csv_path_2020, delimiter=";", on_bad_lines="skip")



Funding distribution comparison (H2020 vs Horizon Europe):
country  ecContribution_2020  netEcContribution_2020  ecContribution_2024  netEcContribution_2024
     DE         1.008683e+10            1.010350e+10         7.143896e+09            7.133945e+09
     FR         7.505264e+09            7.425600e+09         5.005581e+09            4.984713e+09
     ES         6.370864e+09            6.367600e+09         4.525547e+09            4.542529e+09
     IT         5.678991e+09            5.700749e+09         3.810964e+09            3.757276e+09
     NL         5.360284e+09            5.364393e+09         3.791088e+09            3.772894e+09
     BE         3.415569e+09            3.392041e+09         3.394415e+09            3.321351e+09
     EL         1.716628e+09            1.726158e+09         1.677251e+09            1.677521e+09
     SE         2.318316e+09            2.312439e+09         1.459772e+09            1.462400e+09
     NO         1.697087e+09            1.707813e+09      

### In project.csv: column ‘maxEcContribution’

Moeilijkheid: er is geen kolom 'country' in project.csv, daarom kan je organization.csv en project.csv mergen op de id-kolom. Hier zitten nog errors en ik moet er nog naar kijken. Nog op te lossen want nu staan er NaNs

In [19]:
# Paths
proj_path_2020 = "/Users/mariejatsun/Desktop/MDA Project/cordis-h2020projects-csv/project.csv"
org_path_2020  = "/Users/mariejatsun/Desktop/MDA Project/cordis-h2020projects-csv/organization.csv"
proj_path_2024 = "/Users/mariejatsun/Desktop/MDA Project/cordis-HORIZONprojects-csv/project.csv"
org_path_2024  = "/Users/mariejatsun/Desktop/MDA Project/cordis-HORIZONprojects-csv/organization.csv"

# Load CSV's
df_proj_2020 = pd.read_csv(proj_path_2020, delimiter=";", on_bad_lines="skip")
df_org_2020  = pd.read_csv(org_path_2020, delimiter=";", on_bad_lines="skip")
df_proj_2024 = pd.read_csv(proj_path_2024, delimiter=";", on_bad_lines="skip")
df_org_2024  = pd.read_csv(org_path_2024, delimiter=";", on_bad_lines="skip")

# Check if required columns exist
for df, name in [(df_proj_2020, "project.csv 2020"), (df_proj_2024, "project.csv 2024")]:
    if 'ecMaxContribution' not in df.columns:
        raise ValueError(f"Kolom 'ecMaxContribution' ontbreekt in {name}")
if 'id' not in df_proj_2020.columns or 'projectID' not in df_org_2020.columns:
    raise ValueError("Verkeerde kolomnamen in 2020 datasets")
if 'id' not in df_proj_2024.columns or 'projectID' not in df_org_2024.columns:
    raise ValueError("Verkeerde kolomnamen in 2024 datasets")
if 'country' not in df_org_2020.columns or 'country' not in df_org_2024.columns:
    raise ValueError("Kolom 'country' ontbreekt in organization.csv")

# Merge datasets
merged_2020 = pd.merge(
    df_proj_2020[['id', 'ecMaxContribution']],
    df_org_2020[['projectID', 'role', 'country']],
    left_on='id',
    right_on='projectID'
)
merged_2024 = pd.merge(
    df_proj_2024[['id', 'ecMaxContribution']],
    df_org_2024[['projectID', 'role', 'country']],
    left_on='id',
    right_on='projectID'
)

# 
coordinators_2020 = merged_2020[merged_2020['role'] == 'coordinator']
coordinators_2024 = merged_2024[merged_2024['role'] == 'coordinator']

# Som per land
funding_2020 = coordinators_2020.groupby('country')['ecMaxContribution'].sum().reset_index()
funding_2024 = coordinators_2024.groupby('country')['ecMaxContribution'].sum().reset_index()

# Kolomnamen aanpassen
funding_2020.columns = ['country', 'ecMaxContribution_2020']
funding_2024.columns = ['country', 'ecMaxContribution_2024']

# Mergen en printen
merged_funding = pd.merge(funding_2020, funding_2024, on='country', how='outer').fillna(0)
merged_funding['ecMaxContribution_2024'] = pd.to_numeric(merged_funding['ecMaxContribution_2024'], errors='coerce')
merged_funding['ecMaxContribution_2020'] = pd.to_numeric(merged_funding['ecMaxContribution_2020'], errors='coerce')
merged_funding = merged_funding.sort_values(by='ecMaxContribution_2024', ascending=False)


print("\nMax EC Contribution per country (coördinatorprojecten):")
print(merged_funding.to_string(index=False))


/var/folders/xs/sd59f35s4fd7q67ls761c5xm0000gn/T/ipykernel_21906/4160056936.py:9: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_org_2020  = pd.read_csv(org_path_2020, delimiter=";", on_bad_lines="skip")



Max EC Contribution per country (coördinatorprojecten):
country  ecMaxContribution_2020  ecMaxContribution_2024
     GE                     NaN            1.493420e+13
     AU            0.000000e+00            9.989443e+11
     KE            1.000000e+20            5.000000e+06
     AL                     NaN            1.499124e+06
     MK                     NaN            1.329875e+06
     MD                     NaN            2.235000e+05
     AE            5.000000e+05            0.000000e+00
     AI            5.000000e+04            0.000000e+00
     CA            3.000000e+05            0.000000e+00
     CL            5.000000e+04            0.000000e+00
     GL                     NaN            0.000000e+00
     NC            1.288000e+06            0.000000e+00
     US            6.912500e+04            0.000000e+00
     UY            1.355972e+06            0.000000e+00
     AM                     NaN                     NaN
     AT                     NaN                

## Comparison Accross Research Categories

# Research Output accross Countries

# Coordination Roles